In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression

def load_data(file_path, delimiter=' '):
    """
    Loads data from a given file path. Each row is expected to contain 34 numbers,
    where the first 32 numbers are the challenge bits and the last two numbers are the responses.
    """
    data = np.loadtxt(file_path, delimiter=delimiter)
    X = data[:, :32]  # Extract the first 32 columns as the feature matrix
    y0 = data[:, 32]  # Extract the 33rd column as Response0
    y1 = data[:, 33]  # Extract the 34th column as Response1
    return X, y0, y1

def mapped(v):
    a = [0] * 64
    # Calculate a[i] for i from 1 to 31
    for i in range(1, 32):
        sum_val = 0
        for j in range(i, 32):
            prod = 1
            for k in range(i, j + 1):
                prod *= (1 - 2 * v[k-1])  # Adjusted index to match Python's 0-based indexing
            sum_val += v[j] * prod
        a[i-1] = sum_val  # Adjusted index to match Python's 0-based indexing

    # Calculate a[i + 31] = v[i] for i from 1 to 32
    for i in range(1, 33):
        a[i + 30] = v[i-1]  # Adjusted index to match Python's 0-based indexing

    return np.array(a)  # Return as numpy array for compatibility with sklearn

def my_map(X):
    """
    Maps each challenge to a D-dimensional feature vector using the transformation.
    """
    return np.array([mapped(v) for v in X])

def my_fit(X_train, y0_train, y1_train):
    """
    Trains two linear models by invoking an sklearn routine.
    """
    # Transform the training data using the mapping function
    X_mapped = my_map(X_train)

    # Train a linear model for Response0
    model0 = LogisticRegression()
    model0.fit(X_mapped, y0_train)
    W0 = model0.coef_.flatten()
    b0 = model0.intercept_[0]

    # Train a linear model for Response1
    model1 = LogisticRegression()
    model1.fit(X_mapped, y1_train)
    W1 = model1.coef_.flatten()
    b1 = model1.intercept_[0]

    return W0, b0, W1, b1

def get_responses(X, W0, b0, W1, b1):
    """
    Given a set of challenges X and model parameters,
    return the predictions for Response0 and Response1.
    """
    mapped_X = my_map(X)
    linear_combination0 = np.dot(mapped_X, W0) + b0
    linear_combination1 = np.dot(mapped_X, W1) + b1
    responses0 = (linear_combination0 >= 0).astype(int)
    responses1 = (linear_combination1 >= 0).astype(int)
    return responses0, responses1

def evaluate_model(X_test, y_test, W, b):
    """
    Evaluates the model on the test set.
    """
    X_mapped = my_map(X_test)
    predictions = (np.dot(X_mapped, W) + b) >= 0
    accuracy = np.mean(predictions == y_test)
    return accuracy

if __name__ == "__main__":
    # Load training data
    X_train, y0_train, y1_train = load_data('public_trn.txt', delimiter=' ')
    
    # Train the models
    W0, b0, W1, b1 = my_fit(X_train, y0_train, y1_train)
    
    # Load test data
    X_test, y0_test, y1_test = load_data('public_tst.txt', delimiter=' ')
    
    # Get predictions for the test data
    responses0, responses1 = get_responses(X_test, W0, b0, W1, b1)
    
    
    # Evaluate the models and print accuracies
    accuracy0 = evaluate_model(X_test, y0_test, W0, b0)
    accuracy1 = evaluate_model(X_test, y1_test, W1, b1)
    print("Accuracy for Response0:", accuracy0)
    print("Accuracy for Response1:", accuracy1)


Accuracy for Response0: 0.9803
Accuracy for Response1: 0.9953
